# Train YOLO

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")

results = model.train(
        batch=8,
#         device="cpu",
        data="data_refuge2.yaml",
        epochs=1,
        imgsz=640,
    )

# Test

In [ ]:
from IPython.display import Image as show_image
show_image(filename="runs/segment/train_yolo/val_batch0_pred.jpg")

In [ ]:
# Export the model
yolo_model = YOLO( './runs/segment/train3/weights/best.pt' ) 
yolo_model.export(format='onnx')

In [ ]:
import glob
import cv2
import numpy as np
import imutils
from PIL import Image
import numpy
import matplotlib.pyplot as plt
import torchvision.transforms as T

path = 'C:/Users/debora.assis/Documents/REFUGE/test/images'

for image_path in glob.glob(path + "/*"):
    results = list (yolo_model(image_path, conf= 0.1)) 
    result = results[0]
    
    
    if len(result) == 1:
        mask_result = T.ToPILImage()(result.masks.masks)
     
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_name = image_path.split('\\')[1][:-4]
        
        contours_cup = cv2.findContours(np.array(mask_result), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_cup = imutils.grab_contours(contours_cup)

        result  = img.copy()
        result = cv2.resize(img, (224,224))
        for cnt_cup in contours_cup:
            cv2.drawContours(result, [cnt_cup], -1, (0, 255, 0), thickness=1)

                
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 8))

        ax = ax.flatten()
        for a in ax:
            ax[0].imshow(img)
            ax[0].set_title(str(img_name), fontsize=12)
            ax[0].set_xticks([])
            ax[0].set_yticks([])

            ax[1].imshow(mask_result)
            ax[1].set_title('Predic Mask', fontsize=12)
            ax[1].set_xticks([])
            ax[1].set_yticks([])

            ax[2].imshow(result)
            ax[2].set_title('Contour image', fontsize=12)
            ax[2].set_xticks([])
            ax[2].set_yticks([])
        plt.show()
    